In [30]:
import os
import random
from glob import glob

from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

import pandas as pd
import numpy as np


In [2]:
df = pd.read_csv('./../../../russianfood/russianfood_filtred.csv')
df.head(5)

,id,url,title,ingredients,steps,images
0,158659,https://www.russianfood.com/recipes/recipe.php...,Запеканка из куриного филе с грибами и беконом...,Курица (любая часть) - 600 г|Бекон - 100 г|Сме...,Подготовьте все необходимые ингредиенты. Сыр м...,images/158659_0.jpg|images/158659_1.jpg|images...
1,146718,https://www.russianfood.com/recipes/recipe.php...,Сырные лепешки с картошкой (на сухой сковороде),Для теста:|Сыр (твердый или полутвердый) - 250...,"Как приготовить лепешки из сырного теста, с ка...",images/146718_0.jpg|images/146718_1.jpg|images...
2,134548,https://www.russianfood.com/recipes/recipe.php...,"Торт без выпечки, с творогом и черносливом",Печенье песочное (сахарное) – 600 г|Творог – 3...,"Как приготовить торт без выпечки, с творогом и...",images/134548_0.jpg|images/134548_1.jpg|images...
3,151796,https://www.russianfood.com/recipes/recipe.php...,Сырные булочки с помидорами и зелёным луком,Для теста:|Сахар - 1 ст. ложка|Яйцо - 1 шт.|ил...,Для теста будем использовать прессованные дрож...,images/151796_0.jpg|images/151796_1.jpg|images...
4,114586,https://www.russianfood.com/recipes/recipe.php...,Курица в сливках с черносливом,Куриные голени - 4 шт.|Лук репчатый (крупный) ...,Подготовьте все необходимые ингредиенты. Идеал...,images/114586_0.jpg|images/114586_1.jpg|images...


In [3]:
tokenizer = BertTokenizer.from_pretrained("DeepPavlov/rubert-base-cased")
rubert = BertModel.from_pretrained(f'./bert_tuned')
rubert.eval();

Some weights of BertModel were not initialized from the model checkpoint at ./bert_tuned and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
emb_data = []
for index, row in tqdm(df.iterrows(), total=df.shape[0]):

    recipe_id = row['id']
    # title_text = row['title'] 
    title_in = tokenizer(row['title'], return_tensors="pt", max_length=512)
    ingredients_in = tokenizer(row['ingredients'].replace('|', ' '), return_tensors="pt", max_length=512)
    steps_in = tokenizer(row['steps'].replace('|', ' '), return_tensors="pt", max_length=512)

    title = rubert(**title_in)[0][:, 0, :].cpu().detach().numpy()
    ingredients = rubert(**ingredients_in)[0][:, 0, :].cpu().detach().numpy()
    steps = rubert(**steps_in)[0][:, 0, :].cpu().detach().numpy()
    
    np.savez_compressed(f'embeddings/russianfood/title/{recipe_id}.npz', title)
    np.savez_compressed(f'embeddings/russianfood/ingredients/{recipe_id}.npz', ingredients)
    np.savez_compressed(f'embeddings/russianfood/steps/{recipe_id}.npz', steps)

100%|██████████| 13830/13830 [3:35:23<00:00,  1.07it/s]  


In [22]:
df_eda = pd.read_csv('./../../../eda_ru_dataset/eda_ru.csv', sep=';')
df_eda = df_eda[
    (df_eda['images'].notna()) & 
    (df_eda['title'].notna()) & 
    (df_eda['ingredients'].notna()) & 
    (df_eda['steps'].notna())
]
df_eda.head(5)

,id,url,title,ingredients,descr,steps,images,images_step
0,24419,https://eda.ru/recepty/vypechka-deserty/sloeni...,Слоеные булки,Вода:350 мл|Пшеничная мука:500 г|Соль:по вкусу...,NaN,1. Вотрите дрожжи в муку до образования одноро...,images/24419_1.jpg,NaN
1,24410,https://eda.ru/recepty/vypechka-deserty/italja...,Итальянское персиковое мороженое (Gelato),Молоко:6 стаканов|Сахар:1 стакан|Яичный желток...,NaN,1. В блендере измельчите очищенные от кожуры п...,images/24410_1.jpg,NaN
3,24411,https://eda.ru/recepty/vypechka-deserty/italja...,Итальянское мороженое с лимонной цедрой (Gelato),Молоко:6 стаканов|Сахар:1 стакан|Яичный желток...,NaN,1. В большой кастрюле смешайте слегка взбитые ...,images/24411_1.jpg,NaN
4,24414,https://eda.ru/recepty/salaty/zakuska-iz-kuric...,Закуска из курицы в листьях салата,Куриная грудка:1 штука|Оливковое масло:3 столо...,NaN,"1. Смешайте ингредиенты для соуса: масло, соев...",images/24414_1.jpg,NaN
5,24413,https://eda.ru/recepty/zakuski/marinovannie-ku...,Маринованные куриные ломтики,Лайм:1 штука|Чеснок:1 зубчик|Соевый соус:2 сто...,NaN,"1. Приготовьте маринад из сока и цедры лайма, ...",images/24413_1.jpg,NaN


In [28]:
emb_data = []
for index, row in tqdm(df_eda[14700:].iterrows(), total=df_eda[14700:].shape[0]):

    recipe_id = row['id']
    # title_text = row['title'] 
    title_in = tokenizer(row['title'], return_tensors="pt", max_length=512)
    ingredients_in = tokenizer(row['ingredients'].replace('|', ' '), return_tensors="pt", max_length=512)
    steps_in = tokenizer(row['steps'].replace('|', ' '), return_tensors="pt", max_length=512)

    title = rubert(**title_in)[0][:, 0, :].cpu().detach().numpy()
    ingredients = rubert(**ingredients_in)[0][:, 0, :].cpu().detach().numpy()
    steps = rubert(**steps_in)[0][:, 0, :].cpu().detach().numpy()
    
    np.savez_compressed(f'embeddings/eda_ru/title/{recipe_id}.npz', title)
    np.savez_compressed(f'embeddings/eda_ru/ingredients/{recipe_id}.npz', ingredients)
    np.savez_compressed(f'embeddings/eda_ru/steps/{recipe_id}.npz', steps)

100%|██████████| 15391/15391 [3:11:43<00:00,  1.34it/s]  


In [29]:
df_eda.to_csv('./../../../eda_ru_dataset/eda_ru_filtered.csv', index=False)

In [60]:
a1 = np.load('./embeddings/eda_ru/title/104177.npz')['arr_0']
a2 = np.load('./embeddings/eda_ru/ingredients/104177.npz')['arr_0']
a3 = np.load('./embeddings/eda_ru/steps/104177.npz')['arr_0']

In [61]:
np.concatenate([a1, a2, a3], axis=1).shape

(1, 2304)